# Google T5-Small Model Transfer Learning Training on A-100 GPU High RAM (40 GB instance)

## Finetuning

In [ ]:
!pip install transformers datasets rouge-score nltk evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=88f7d5764f21a1c0f31c1cd9780fc87b02a0aafba43bb25a6ea82738618ea98d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's depen

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# Load the dataset
dataset = load_dataset("ccdv/pubmed-summarization")

# Load the pre-trained model and tokenizer
model_name = "Falconsai/text_summarization"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


In [ ]:
train_samples = 3000  # Number of training samples
val_samples = 300    # Number of validation samples
test_samples = 300   # Number of test samples

In [ ]:
# Function to randomly select a specific number of samples
def get_subset(dataset_split, num_samples):
    # Shuffle the dataset and select the first `num_samples`
    shuffled_dataset = dataset_split.shuffle(seed=42)  # Seed for reproducibility
    subset = shuffled_dataset.select(range(num_samples))
    return subset

# Get subsets for train, validation, and test
train_subset = get_subset(dataset["train"], train_samples)
val_subset = get_subset(dataset["validation"], val_samples)
test_subset = get_subset(dataset["test"], test_samples)

# Create a new DatasetDict with the subsets
dataset = DatasetDict({
    "train": train_subset,
    "validation": val_subset,
    "test": test_subset
})

In [ ]:
def preprocess_data(examples):
    # Tokenize inputs with padding and truncation
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(
        inputs,
        max_length=5096,  # Maximum length for input sequences
        truncation=True,  # Truncate sequences longer than max_length
        padding="max_length",  # Pad sequences to max_length
    )

    # Tokenize labels with padding and truncation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["abstract"],
            max_length=512,  # Maximum length for label sequences
            truncation=True,  # Truncate sequences longer than max_length
            padding="max_length",  # Pad sequences to max_length
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }
    for pred, label in zip(decoded_preds, decoded_labels):
        scores = scorer.score(label, pred)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    bleu_scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        bleu_scores.append(sentence_bleu([label.split()], pred.split()))

    return {
        'pred':decoded_preds
    }

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

<ipython-input-11-caf68606dd06>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


TrainOutput(global_step=15000, training_loss=1.4051358072916666, metrics={'train_runtime': 8905.0847, 'train_samples_per_second': 3.369, 'train_steps_per_second': 1.684, 'total_flos': 4.041221603328e+16, 'train_loss': 1.4051358072916666, 'epoch': 10.0})

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-text-summarization")
tokenizer.save_pretrained("./fine-tuned-text-summarization")

Trainer is attempting to log a value of "['the birth of a premature infant and the following neonatal intensive care unit ( nicu', 'we propose that a definition of health ( nor treatment ) is regulated in a similar', '', '', 'objective : the purpose of this review is to present updated data available on the efficacy', 'objective : the aim of this study was to investigate the effect of varied protein - energy combinations', 'tcr mice ( 21 ) were mated with rag2 mice ( 23', 'background : sprayed refuges are a compromise between sprayed and unsprayed refuge', 'the hypnotic assessment method was based on the prolongation of sleep induced by pen', 'succinic acid is classified as the most promising chemical among 12 biobased chemicals by the u', '', 'this review was based on a systematic comprehensive search of six databases : ovid', '', '', '', 'axonal excitability is a major determinant of the treatment regimen required to', 'neurological soft signs ( nss ) have long been considered one of

{'eval_loss': 1.2174769639968872, 'eval_pred': ['the birth of a premature infant and the following neonatal intensive care unit ( nicu', 'we propose that a definition of health ( nor treatment ) is regulated in a similar', '', '', 'objective : the purpose of this review is to present updated data available on the efficacy', 'objective : the aim of this study was to investigate the effect of varied protein - energy combinations', 'tcr mice ( 21 ) were mated with rag2 mice ( 23', 'background : sprayed refuges are a compromise between sprayed and unsprayed refuge', 'the hypnotic assessment method was based on the prolongation of sleep induced by pen', 'succinic acid is classified as the most promising chemical among 12 biobased chemicals by the u', '', 'this review was based on a systematic comprehensive search of six databases : ovid', '', '', '', 'axonal excitability is a major determinant of the treatment regimen required to', 'neurological soft signs ( nss ) have long been considered 

('./fine-tuned-text-summarization-specific-samples/tokenizer_config.json',
 './fine-tuned-text-summarization-specific-samples/special_tokens_map.json',
 './fine-tuned-text-summarization-specific-samples/spiece.model',
 './fine-tuned-text-summarization-specific-samples/added_tokens.json')

In [ ]:
!zip -r ./fine-tuned-text-summarization.zip ./fine-tuned-text-summarization

  adding: content/fine-tuned-text-summarization-specific-samples/ (stored 0%)
  adding: content/fine-tuned-text-summarization-specific-samples/spiece.model (deflated 48%)
  adding: content/fine-tuned-text-summarization-specific-samples/config.json (deflated 62%)
  adding: content/fine-tuned-text-summarization-specific-samples/special_tokens_map.json (deflated 85%)
  adding: content/fine-tuned-text-summarization-specific-samples/tokenizer_config.json (deflated 94%)
  adding: content/fine-tuned-text-summarization-specific-samples/model.safetensors (deflated 9%)
  adding: content/fine-tuned-text-summarization-specific-samples/added_tokens.json (deflated 83%)
  adding: content/fine-tuned-text-summarization-specific-samples/generation_config.json (deflated 29%)


## Benchmarking against CompScholar


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e755c5e6c1951487b4ec4488f7206b29736e820fb86136aea6d11506673cb0ce
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
!unzip ./fine-tuned-text-summarization.zip

Archive:  /content/fine-tuned-text-summarization-specific-samples-version7_og.zip
   creating: content/fine-tuned-text-summarization-specific-samples/
  inflating: content/fine-tuned-text-summarization-specific-samples/special_tokens_map.json  
  inflating: content/fine-tuned-text-summarization-specific-samples/config.json  
  inflating: content/fine-tuned-text-summarization-specific-samples/tokenizer_config.json  
  inflating: content/fine-tuned-text-summarization-specific-samples/model.safetensors  
  inflating: content/fine-tuned-text-summarization-specific-samples/generation_config.json  
  inflating: content/fine-tuned-text-summarization-specific-samples/spiece.model  
  inflating: content/fine-tuned-text-summarization-specific-samples/added_tokens.json  


In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
import warnings
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/content/fine-tuned-text-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/content/fine-tuned-text-summarization")
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

Device set to use cpu


In [ ]:
df = pd.read_csv('/content/Brain Dead CompScholar Dataset.csv')

In [ ]:
def calculate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

def calculate_bleu(reference, candidate):
    smoothie = SmoothingFunction().method1
    reference_tokens = nltk.word_tokenize(reference.lower())
    candidate_tokens = nltk.word_tokenize(candidate.lower())

    return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothie)


In [ ]:
results = []

print("Processing documents and evaluating...")
for _, row in df.iterrows():
    paper_id = row['Paper Id']
    document = row['Document']
    reference_summary = row['Summary']

    try:
        print(f"Generating summary for Paper ID: {paper_id}")
        generated_summary = summarizer(document, max_length=300, min_length=90, do_sample=False)
        generated_summary_text = generated_summary[0]['summary_text']

        rouge_scores = calculate_rouge(reference_summary, generated_summary_text)
        bleu_score = calculate_bleu(reference_summary, generated_summary_text)

        results.append({
            'Paper_Id': paper_id,
            'Reference_Summary': reference_summary,
            'Generated_Summary': generated_summary_text,
            'ROUGE-1': rouge_scores['rouge1'],
            'ROUGE-2': rouge_scores['rouge2'],
            'ROUGE-L': rouge_scores['rougeL'],
            'BLEU': bleu_score
        })

        print(f"Processed Paper ID: {paper_id}")
    except Exception as e:
        print(f"Error processing Paper ID {paper_id}: {str(e)}")



Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors


Processing documents and evaluating...
Generating summary for Paper ID: 1
Processed Paper ID: 1
Generating summary for Paper ID: 2
Processed Paper ID: 2
Generating summary for Paper ID: 3
Processed Paper ID: 3
Generating summary for Paper ID: 4
Processed Paper ID: 4
Generating summary for Paper ID: 5
Processed Paper ID: 5
Generating summary for Paper ID: 6
Processed Paper ID: 6
Generating summary for Paper ID: 7
Processed Paper ID: 7
Generating summary for Paper ID: 8
Processed Paper ID: 8
Generating summary for Paper ID: 9
Processed Paper ID: 9
Generating summary for Paper ID: 10
Processed Paper ID: 10
Generating summary for Paper ID: 11
Processed Paper ID: 11
Generating summary for Paper ID: 12
Processed Paper ID: 12
Generating summary for Paper ID: 13
Processed Paper ID: 13
Generating summary for Paper ID: 14
Processed Paper ID: 14
Generating summary for Paper ID: 15
Processed Paper ID: 15
Generating summary for Paper ID: 16
Processed Paper ID: 16
Generating summary for Paper ID: 17

Your max_length is set to 300, but your input_length is only 285. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=142)


Processed Paper ID: 186
Generating summary for Paper ID: 187
Processed Paper ID: 187
Generating summary for Paper ID: 188
Processed Paper ID: 188
Generating summary for Paper ID: 189
Processed Paper ID: 189
Generating summary for Paper ID: 190
Processed Paper ID: 190
Generating summary for Paper ID: 191
Processed Paper ID: 191
Generating summary for Paper ID: 192
Processed Paper ID: 192
Generating summary for Paper ID: 193
Processed Paper ID: 193
Generating summary for Paper ID: 194
Processed Paper ID: 194
Generating summary for Paper ID: 195
Processed Paper ID: 195
Generating summary for Paper ID: 196
Processed Paper ID: 196
Generating summary for Paper ID: 197
Processed Paper ID: 197
Generating summary for Paper ID: 198
Processed Paper ID: 198
Generating summary for Paper ID: 199
Processed Paper ID: 199
Generating summary for Paper ID: 200


Your max_length is set to 300, but your input_length is only 295. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=147)


...

Processed Paper ID: 344
Generating summary for Paper ID: 345
Processed Paper ID: 345
Generating summary for Paper ID: 346
Processed Paper ID: 346
Generating summary for Paper ID: 347


Your max_length is set to 300, but your input_length is only 237. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)


Processed Paper ID: 347
Generating summary for Paper ID: 348


Your max_length is set to 300, but your input_length is only 215. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=107)


Processed Paper ID: 348
Generating summary for Paper ID: 349
Processed Paper ID: 349
Generating summary for Paper ID: 350
Processed Paper ID: 350
Generating summary for Paper ID: 351
Processed Paper ID: 351
Generating summary for Paper ID: 352
Processed Paper ID: 352
Generating summary for Paper ID: 353
Processed Paper ID: 353
Generating summary for Paper ID: 354
Processed Paper ID: 354
Generating summary for Paper ID: 355
Processed Paper ID: 355
Generating summary for Paper ID: 356
Processed Paper ID: 356
Generating summary for Paper ID: 357
Processed Paper ID: 357
Generating summary for Paper ID: 358
Processed Paper ID: 358
Generating summary for Paper ID: 359
Processed Paper ID: 359
Generating summary for Paper ID: 360
Processed Paper ID: 360
Generating summary for Paper ID: 361
Processed Paper ID: 361
Generating summary for Paper ID: 362
Processed Paper ID: 362
Generating summary for Paper ID: 363
Processed Paper ID: 363
Generating summary for Paper ID: 364
Processed Paper ID: 364


In [ ]:
results_df = pd.DataFrame(results)

# Multiply the scores by 100 to scale them to 0-100 from 0-1
results_df['ROUGE-1'] = results_df['ROUGE-1'] * 100
results_df['ROUGE-2'] = results_df['ROUGE-2'] * 100
results_df['ROUGE-L'] = results_df['ROUGE-L'] * 100
results_df['BLEU'] = results_df['BLEU'] * 100

avg_scores = {
    'ROUGE-1': results_df['ROUGE-1'].mean(),
    'ROUGE-2': results_df['ROUGE-2'].mean(),
    'ROUGE-L': results_df['ROUGE-L'].mean(),
    'BLEU': results_df['BLEU'].mean()
}

print("\nSummary Statistics:")
for metric, score in avg_scores.items():
    print(f"Average {metric}: {score:.4f}")


print("\nDetailed Results:")
print(results_df[['Paper_Id', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU']])

results_df.to_csv('CompScholar_summarization_evaluation_results.csv', index=False)
print("\nResults saved to 'CompScholar_summarization_evaluation_results.csv'")


Summary Statistics:
Average ROUGE-1: 46.2173
Average ROUGE-2: 21.4349
Average ROUGE-L: 29.5130
Average BLEU: 12.3004

Detailed Results:
     Paper_Id    ROUGE-1    ROUGE-2    ROUGE-L       BLEU
0           1  61.635220  24.203822  32.704403  15.710743
1           2  37.777778   5.617978  23.333333   2.222723
2           3  48.000000  22.972973  36.000000  14.858638
3           4  50.370370  24.060150  34.074074   9.384487
4           5  50.322581  22.222222  29.677419  10.218584
..        ...        ...        ...        ...        ...
366       367  44.615385  14.062500  29.230769   9.388698
367       368  45.962733  12.578616  26.086957   3.218976
368       369  46.575342  12.500000  30.136986   7.834416
369       370  44.970414  11.976048  20.118343   2.474897
370       371  50.000000  20.253165  35.000000  18.891116

[371 rows x 5 columns]

Results saved to 'summarization_evaluation_results.csv'
